In [ ]:
import matplotlib.pyplot as plt
import gzip
import numpy as np
from numpy.linalg import *
from matplotlib import rcParams
from sklearn import linear_model


rcParams.update({'font.size': 18})

In [ ]:
%matplotlib inline  

In [ ]:
#pick number to look at
number = 2

In [ ]:
#functions that are used throughout 
def find_results(data, x):
    #Takes the data and the x results of a fit 
    f = np.matmul(data, x)
    f_max = np.zeros(len(f))
    for i in range(len(f)):
        if(f[i]) > 0.5:
            f_max[i] = 1
    return f, f_max

def find_error(data, labels, x):
    #finds the error in the results and the maximized results.
    f, f_max = find_results(data, x)
    f_error = np.linalg.norm(labels - f, ord=2)/np.linalg.norm(labels, ord=2)
    f_max_error = np.linalg.norm(labels - f_max, ord=2)/np.linalg.norm(labels, ord=2)
    return f_error, f_max

def find_num_right(x, data, labels):
    f,f_max = find_results(data, x)
    num_right = 0
    looking = f_max - labels
#     print(looking)
    for i in range(len(f)):
#         print(looking[i])
        if looking [i] == 0:
            num_right += 1
    return num_right


def count_num_zero(x):
    #counts the number of elements that have zero for all 10
    num_zero= 0
    for i in range(len(x)):
        if(x[i])==0:
            num_zero += 1
    return num_zero

In [ ]:

#open files
f = gzip.open('train-images-idx3-ubyte.gz','r')

image_size = 28
num_images = 60000 


print(image_size * image_size * num_images)
f.read(16)
buf = f.read(image_size * image_size * num_images)
data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
print(len(data))
print(len(buf))
data = data.reshape(num_images, image_size* image_size)

print(data.shape)


In [ ]:
f = gzip.open('train-labels-idx1-ubyte.gz','r')
f.read(8)
buf = f.read(image_size * image_size * num_images)
labels_num = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)

# print(labels_num[3:10])
print(labels_num.shape)

#set_up b vector right
labels = np.zeros([num_images])
# print(labels[1][1])
for i in range(num_images):
        if labels_num[i] == number:
            index = int(number - 1)
            labels[i] = 1
print(labels)

In [ ]:
#get test data
f = gzip.open('t10k-images-idx3-ubyte.gz','r')

# image_size = 28
num_images_test = 10000 
f.read(16)
buf = f.read(image_size * image_size * num_images_test)
test_data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
test_data = test_data.reshape(num_images_test, image_size* image_size)

print(test_data.shape)

f = gzip.open('t10k-labels-idx1-ubyte.gz','r')
f.read(8)
buf = f.read(image_size * image_size * num_images_test)
labels_num = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)

# print(labels_num[3:10])
# print(labels_num.shape)
from collections import Counter
print(Counter(labels_num))
#set_up b vector right
test_labels = np.zeros([num_images_test])
# print(labels[1][1])
for i in range(num_images_test):
    if int(labels_num[i]) == number:
            test_labels[i] = 1
print(test_labels.shape)
print(Counter(test_labels))

In [ ]:
def predict_lasso(alpha, data, labels, test  = False):
    print(alpha)
    clf = linear_model.Lasso(alpha=alpha)
    clf.fit(data, labels)
    x = clf.coef_
    x = np.transpose(x)
    print(x.shape)
    num_zero = count_num_zero(x)
    num_right = find_num_right(x, data, labels)
    if test:
            num_right_test = find_num_right(x, test_data, test_labels)
            return num_zero, num_right, num_right_test, x
    
    return num_zero, num_right, x


def many_lasso(num, data, labels, lower = 0.1, upper = 1, plot=True):
    alpha_list = []
    right_list = []
    num_zero_list = []
    for alpha in np.linspace(lower, upper, num):
        print(alpha)
        num_zero, num_right = predict_lasso(alpha, data, labels)
        right_list.append(num_right)
        num_zero_list.append(num_zero)
        alpha_list.append(alpha)
        
    if plot:
        plt.plot(num_zero_list, right_list, 'ob')
#         plt.plot(num_zero_list, error_list, 'or')

    return alpha_list, right_list, num_zero_list

def many_lasso_test(num, data, labels, lower = 0.1, upper = 1, plot=True):
    alpha_list = []
    right_list = []
    num_zero_list = []
    right_test_list = []
    x_list = []
    for alpha in np.logspace(lower, upper, num = num):
        print(alpha)
        num_zero, num_right, num_right_test, x = predict_lasso(alpha, data, labels, test = True)
        right_list.append(num_right)
        num_zero_list.append(num_zero)
        alpha_list.append(alpha)
        right_test_list.append(num_right_test)
        x_list.append(x)
        
    try:
        if plot:
            plt.plot(num_zero_list, right_list, 'ob')
            plt.plot(num_zero_list, right_test_list, 'or')
    except:
        print("Problem plotting")

    return alpha_list, right_list, num_zero_list, right_test_list, x_list

In [ ]:
np.logspace(-5, 1.17, num = 20)

In [ ]:
alpha_list, right_list, num_zero_list, right_test_list, x_list = many_lasso_test(10, data, labels, lower = -5, upper = 1.17, plot = False)

In [ ]:
x_axis = np.ones(len(num_zero_list)) *image_size*image_size - num_zero_list
plt.plot(x_axis, np.divide(right_list, sum(labels)), 'ob', label = "Training Data")
plt.plot(x_axis, np.divide(right_test_list, sum(test_labels)), 'or', label= "Testing Data")
plt.xlabel("Number of nonzero terms")
plt.ylabel("% predicted correctly")
plt.legend()
plt.title("Predicting occurence of "+str(int(number))+"s with varying number of terms")
plt.show()

In [ ]:
x_axis
print(right_list[-1]/60000)

In [ ]:
num_zero, num_right, num_right_test, x = predict_lasso(alpha_list[5], data, labels, test = True)

In [ ]:
print(num_right / 60000)

In [ ]:
#plotting multiple figures

fig, axs = plt.subplots(2, 2,  constrained_layout=True)
i = 2
x = x_list[i]
# normed = norm(x, axis = 1)
box = x.reshape(image_size, image_size)
# x1_norm = norm(x1_box, axis = 2)
c = axs[0,0].pcolor(box)
axs[0,0].set_title("Nonzero Terms = "+str(image_size**2 - num_zero_list[i]))
fig.colorbar(c, ax=axs[0,0])

i = 6
x = x_list[i]
# normed = norm(x, axis = 1)
box = x.reshape(image_size, image_size)
# x1_norm = norm(x1_box, axis = 2)
c = axs[0,1].pcolor(box)
axs[0,1].set_title("Nonzero Terms = "+str(image_size**2 - num_zero_list[i]))
fig.colorbar(c, ax=axs[0,1])

i = 7
x = x_list[i]
# normed = norm(x, axis = 1)
box = x.reshape(image_size, image_size)
# x1_norm = norm(x1_box, axis = 2)
c = axs[1,0].pcolor(box)
axs[1,0].set_title("Nonzero Terms = "+str(image_size**2 - num_zero_list[i]))
fig.colorbar(c, ax=axs[1,0])

i = -2
x = x_list[i]
# normed = norm(x, axis = 1)
box = x.reshape(image_size, image_size)
# x1_norm = norm(x1_box, axis = 2)
c = axs[1,1].pcolor(box)
axs[1,1].set_title("Nonzero Terms = "+str(image_size**2 - num_zero_list[i]))
fig.colorbar(c, ax=axs[1,1])

plt.suptitle("Importance of Each pixel for predicting "+str(int(number))+"s with various number of nonzero terms")
# plt.tight_layout()
